### Protein Family Classification

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
family_classification_metadata = pd.read_table('../seminar_5/data/family_classification_metadata.tab')
family_classification_sequences = pd.read_table('../seminar_5/data/family_classification_sequences.tab')

In [4]:
family_classification_metadata.head()

,SwissProtAccessionID,LongID,ProteinName,FamilyID,FamilyDescription
0,Q6GZX4,001R_FRG3G,Putative transcription factor 001R,Pox_VLTF3,Poxvirus Late Transcription Factor VLTF3 like
1,Q6GZX3,002L_FRG3G,Uncharacterized protein 002L,DUF230,Poxvirus proteins of unknown function
2,Q6GZX0,005R_FRG3G,Uncharacterized protein 005R,US22,US22 like
3,Q91G88,006L_IIV6,Putative KilA-N domain-containing protein 006L,DUF3627,Protein of unknown function (DUF3627)
4,Q197F3,007R_IIV3,Uncharacterized protein 007R,DUF2738,Protein of unknown function (DUF2738)


In [5]:
family_classification_sequences.head()

,Sequences
0,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...
1,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...
2,MQNPLPEVMSPEHDKRTTTPMSKEANKFIRELDKKPGDLAVVSDFV...
3,MDSLNEVCYEQIKGTFYKGLFGDFPLIVDKKTGCFNATKLCVLGGK...
4,MEAKNITIDNTTYNFFKFYNINQPLTNLKYLNSERLCFSNAVMGKI...


In [6]:
family_classification_metadata.describe()

,SwissProtAccessionID,LongID,ProteinName,FamilyID,FamilyDescription
count,324018,324018,324018,324018,324018
unique,287308,295671,56951,7027,6967
top,Q1X881,POLG_TBEVH,UvrABC system protein B,MMR_HSR1,50S ribosome-binding GTPase
freq,16,12,1500,3084,3084


#### Task:
    
Use your ProtVec embedding from homework 5 to perform protein family classification using RNN.

Article with the original research can be found here http://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0141287&type=printable

* use 1000 most frequent families for classification
* validate your results on the train-test split
* reduce the dimensionality of the protein-space using Stochastic Neighbor Embedding and visualize two most frequent classes
* compare your RNN results with SVM
* visualization and metrics are up to you

In [7]:
NUMBER_OF_FAMILIES = 1000
MAX_PROTEIN_LENGTH = 500
def get_families(size=NUMBER_OF_FAMILIES):
    counter = {}
    for id in family_classification_metadata["FamilyID"]:
        if id not in counter:
            counter[id] = 0
        counter[id] += 1
    
    sorted_families = sorted(counter, key=counter.__getitem__, reverse=True)
    return sorted_families[:size]
            

In [8]:
families = get_families()
families_map = {fam: i for i, fam in enumerate(families)}

In [9]:
def process_prot2vec_from_article():
    res_dict = {}
    for line in pd.read_csv('../seminar_6/data/protVec_100d_3grams.csv').as_matrix():
        split_line = line[0].split()
        res_dict[split_line[0]] = np.array([float(x) for x in split_line[1:]])
    return res_dict

In [10]:
prot2vec = process_prot2vec_from_article()

In [11]:
from lazy import lazy

class SequenceClassificationModel:
    def __init__(self, params):
        self.params = params
        self._create_placeholders()
        self.prediction
        self.cost
        self.error
        self.optimize
        self._create_summaries()
    
    def _create_placeholders(self):
        with tf.name_scope("data"):
            self.data = tf.placeholder(tf.float32, [None, self.params.seq_length, self.params.embed_length])
            self.target = tf.placeholder(tf.float32, [None, NUMBER_OF_FAMILIES])
  
    def _create_summaries(self):
        with tf.name_scope("summaries"):
            tf.summary.scalar('loss', self.cost)
            tf.summary.scalar('erroe', self.error)
            self.summary = tf.summary.merge_all()
            saver = tf.train.Saver()
            
    @lazy
    def length(self):
    # First, we obtain the lengths of sequences in the current data batch. We need this since
    # the data comes as a single tensor, padded with zero vectors to the longest review length.
    # Instead of keeping track of the sequence lengths of every review, we just compute it
    # dynamically in TensorFlow.
    
        with tf.name_scope("seq_length"):
            used = tf.sign(tf.reduce_max(tf.abs(self.data), reduction_indices=2))
            length = tf.reduce_sum(used, reduction_indices=1)
            length = tf.cast(length, tf.int32)
        return length
    
    @lazy
    def prediction(self):
    # Note that the last relevant output activation of the RNN has a different index for each
    # sequence in the training batch. This is because each review has a different length. We
    # already know the length of each sequence.
    # The problem is that we want to index in the dimension of time steps, which is
    # the second dimension in the batch of shape  (sequences, time_steps, word_vectors) .
    
        with tf.name_scope("recurrent_layer"):
            output, _ = tf.nn.dynamic_rnn(
                self.params.rnn_cell(self.params.rnn_hidden),
                self.data,
                dtype=tf.float32,
                sequence_length=self.length
            )
        last = self._last_relevant(output, self.length)

        with tf.name_scope("softmax_layer"):
            num_classes = int(self.target.get_shape()[1])
            weight = tf.Variable(tf.truncated_normal(
                [self.params.rnn_hidden, num_classes], stddev=0.01))
            bias = tf.Variable(tf.constant(0.1, shape=[num_classes]))
            prediction = tf.nn.softmax(tf.matmul(last, weight) + bias)
        return prediction
    
    @lazy
    def cost(self):
        cross_entropy = -tf.reduce_sum(self.target * tf.log(self.prediction))
        return cross_entropy
    
    @lazy
    def error(self):
        self.mistakes = tf.not_equal(
            tf.argmax(self.target, 1), tf.argmax(self.prediction, 1))
        return tf.reduce_mean(tf.cast(self.mistakes, tf.float32))
    
    @lazy
    def optimize(self):
    # RNNs are quite hard to train and weights tend to diverge if the hyper parameters do not
    # play nicely together. The idea of gradient clipping is to restrict the the values of the
    # gradient to a sensible range. This way, we can limit the maximum weight updates.

        with tf.name_scope("optimization"):
            gradient = self.params.optimizer.compute_gradients(self.cost)
            if self.params.gradient_clipping:
                limit = self.params.gradient_clipping
                gradient = [
                    (tf.clip_by_value(g, -limit, limit), v)
                    if g is not None else (None, v)
                    for g, v in gradient]
            optimize = self.params.optimizer.apply_gradients(gradient)
        return optimize
    
    @staticmethod
    def _last_relevant(output, length):
        with tf.name_scope("last_relevant"):
            # As of now, TensorFlow only supports indexing along the first dimension, using
            # tf.gather() . We thus flatten the first two dimensions of the output activations from their
            # shape of  sequences x time_steps x word_vectors  and construct an index into this resulting tensor.
            batch_size = tf.shape(output)[0]
            max_length = int(output.get_shape()[1])
            output_size = int(output.get_shape()[2])

            # The index takes into account the start indices for each sequence in the flat tensor and adds
            # the sequence length to it. Actually, we only add  length - 1  so that we select the last valid
            # time step.
            index = tf.range(0, batch_size) * max_length + (length - 1)
            flat = tf.reshape(output, [-1, output_size])
            relevant = tf.gather(flat, index)
        return relevant

In [12]:
def parse(protein):
    embedded = []
    for i in range(0, len(protein) - 3, 3):
        try:
            embedded.append(prot2vec[protein[i:i+3]])
        except KeyError:
            embedded.append(prot2vec['<unk>'])
    return np.array(embedded)
    
class Embedding:
    def __init__(self, length, parser=parse):
        self.parser = parser
        self._length = length
        self.dimensions = 100
        
    def __call__(self, protein):
        embedded = self.parser(protein)
        if len(embedded) < self._length:
            embedded = np.vstack((embedded, np.zeros(shape=(self._length - embedded.shape[0], self.dimensions))))
        return np.array(embedded)

In [13]:
proteins = np.array(family_classification_sequences['Sequences'])
families = np.array(family_classification_metadata['FamilyID'])

length = MAX_PROTEIN_LENGTH

embedding = Embedding(length)

In [14]:
from attrdict import AttrDict

params = AttrDict(
    rnn_cell=tf.contrib.rnn.GRUCell,
    rnn_hidden=256,
    optimizer=tf.train.AdamOptimizer(0.001),
    batch_size=256,
    gradient_clipping=100,
    seq_length=length,
    embed_length=embedding.dimensions
)

In [15]:
def build_target(family):
    return np.eye(NUMBER_OF_FAMILIES)[families_map[family]]
    
def preprocess_batched(iterator, length, embedding, batch_size):
    while True:
        data = []
        target = []
        for index in range(batch_size):
            try:
                protein, family = next(iterator)
            except:
                return
            data.append(embedding(protein))
            target.append(build_target(family))
        yield np.array(data), np.array(target)

In [16]:
valid_indices = []
for i,f in enumerate(families):
    if f not in families_map or len(proteins[i]) > MAX_PROTEIN_LENGTH:
        continue
    valid_indices.append(i)
valid_indices = np.array(valid_indices)
np.random.shuffle(valid_indices)

train_ratio = 0.9

train_indices = valid_indices[:int(valid_indices.shape[0] * train_ratio)]
test_indices = valid_indices[int(valid_indices.shape[0] * train_ratio):]

print(len(train_indices))
print(len(test_indices))

177226
19692


In [17]:
def train_iterator():
    for index in train_indices:
        yield (proteins[index], families[index])
    
def test_iterator():
    for index in test_indices:
        yield (proteins[index], families[index])


In [18]:
tf.reset_default_graph()

model = SequenceClassificationModel(params)

/opt/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.25)

batches = preprocess_batched(train_iterator(), length, embedding, params.batch_size)
iterations = 10000
with tf.Session(config=tf.ConfigProto(gpu_options=gpu_options)) as sess:
    
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter('graphs', sess.graph)

    for index, batch in enumerate(batches):
        feed = {model.data: batch[0], model.target: batch[1]}
        error, _, summary_str = sess.run([model.error, model.optimize, model.summary], feed)
        print('{}: {:3.1f}%'.format(index + 1, 100 * error))
        if index % 1 == 0:
            summary_writer.add_summary(summary_str, index)
        if index == iterations:
            break
    
    sum_error = 0.
    counter = 0.
    
    testing = preprocess_batched(test_iterator(), length, embedding, params.batch_size)
    for index, batch in enumerate(testing):
        feed = {model.data: batch[0], model.target: batch[1]}
        error = sess.run(model.error, feed)
        sum_error += error
        counter += 1
        print('Accuracy on testing: {:3.1f}%'.format(100 * (1-sum_error/counter)))

1: 99.6%
2: 100.0%
3: 99.6%
4: 100.0%
5: 99.6%
6: 99.2%
7: 98.8%
8: 98.0%
9: 98.0%
10: 98.8%
11: 97.7%
12: 97.7%
13: 98.8%
14: 98.8%
15: 99.2%
16: 98.0%
17: 98.4%
18: 97.3%
19: 97.7%
20: 98.4%
21: 99.2%
22: 98.4%
23: 98.8%
24: 98.8%
25: 98.4%
26: 98.4%
27: 98.8%
28: 98.8%
29: 98.8%
30: 98.8%
31: 99.6%
32: 100.0%
33: 99.6%
34: 98.8%
35: 99.6%
36: 98.8%
37: 99.2%
38: 99.2%
39: 99.6%
40: 98.8%
41: 98.8%
42: 98.4%
43: 97.7%
44: 98.4%
45: 99.2%
46: 98.8%
47: 99.2%
48: 99.2%
49: 100.0%
50: 98.8%
51: 98.4%
52: 97.3%
53: 99.2%
54: 98.4%
55: 98.4%
56: 98.4%
57: 99.2%
58: 98.4%
59: 97.7%
60: 97.7%
61: 99.2%
62: 98.4%
63: 98.4%
64: 98.4%
65: 98.8%
66: 98.4%
67: 97.7%
68: 98.0%
69: 98.4%
70: 96.9%
71: 99.6%
72: 98.8%
73: 99.2%
74: 98.4%
75: 98.0%
76: 99.6%
77: 97.7%
78: 98.0%
79: 97.7%
80: 98.0%
81: 96.9%
82: 96.9%
83: 98.0%
84: 98.8%
85: 98.0%
86: 97.7%
87: 97.7%
88: 97.3%
89: 97.7%
90: 97.7%
91: 97.7%
92: 97.7%
93: 97.7%
94: 98.0%
95: 98.8%
96: 98.0%
97: 96.9%
98: 98.0%
99: 96.5%
100: 98.4%
101:

In [ ]:
print('Accuracy on testing: {:3.1f}%'.format(100 * (1-sum_error/counter)))

In [20]:
def sk_parse(protein):
    embedded = []
    for i in range(0, len(protein) - 3, 3):
        try:
            embedded.append(prot2vec[protein[i:i+3]])
        except KeyError:
            embedded.append(prot2vec['<unk>'])
    res_embedded = embedded[0]
    for i in range(1, len(embedded)):
        res_embedded += embedded[i]
    res_embedded = np.array(res_embedded)
    res_embedded /= len(res_embedded)
    return np.array(res_embedded)

sk_embedding = Embedding(length, parser=sk_parse)

In [ ]:
def get_proteins_sk(iterator, length, sk_embedding, batch_size):
    while True:
        data = []
        for index in range(batch_size):
            try:
                protein, family = next(iterator)
            except:
                return
            data.append(sk_embedding(protein))
        yield np.array(data)
def get_families_sk(iterator, length, sk_embedding, batch_size):
    while True:
        target = []
        for index in range(batch_size):
            try:
                protein, family = next(iterator)
            except:
                return
            target.append(families_map[family])
        yield np.array(target)

In [ ]:
sk_train_prot = list(get_proteins_sk(train_iterator(), length, sk_embedding, params.batch_size))
sk_train_fams = list(get_families_sk(train_iterator(), length, sk_embedding, params.batch_size))
sk_test_prot = get_proteins_sk(test_iterator(), length, sk_embedding, params.batch_size)
sk_test_fams = get_families_sk(test_iterator(), length, sk_embedding, params.batch_size)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in add
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(n_estimators=16, max_depth=16)
clf.fit(sk_train_prot, sk_train_fams)